In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch # type: ignore
import sys; sys.path.append('../')
from  giagrad.tensor import Tensor
import numpy as np
from giagrad.display import draw_dot
import torch.nn as nn
import string
import math
from numpy.lib.stride_tricks import as_strided

In [117]:
k = np.random.randint(0, 4,(3, 2, 2, 2))
k

array([[[[1, 2],
         [3, 0]],

        [[2, 3],
         [2, 3]]],


       [[[2, 2],
         [2, 3]],

        [[2, 0],
         [1, 0]]],


       [[[0, 1],
         [2, 1]],

        [[1, 3],
         [1, 2]]]])

In [111]:
import string
alphabet = string.ascii_letters + '0123456789!@#$%^&*()_+=-'
# a = np.arange(0, 2*2*3*3).reshape((2, 2, 3, 3))
a = np.array(list(alphabet[:1*2*3*3])).reshape((1, 2, 3, 3)).astype('object')
a

array([[[['a', 'b', 'c'],
         ['d', 'e', 'f'],
         ['g', 'h', 'i']],

        [['j', 'k', 'l'],
         ['m', 'n', 'o'],
         ['p', 'q', 'r']]]], dtype=object)

Se necesita convertir con as_strided el array anterior para que tenga dimension:
    $$(N, C_{in}, H_{in}, W_{in})$$
Donde 
$$
H_{out} = \left\lfloor 
            \frac{H_{in} + 2 \times padding[0] - dilation[0] \times (kernel\_size[0] -1) - 1}
                 {stride[0]} + 1
           \right\rfloor \\
W_{out} = \left\lfloor 
            \frac{W_{in} + 2 \times padding[0] - dilation[0] \times (kernel\_size[1] -1) - 1}
                 {stride[1]} + 1 
           \right\rfloor \\
$$

In [112]:
from math import floor

N, Cin, Hin, Win = a.shape
k1, k2 = k.shape[-2:]
Cout = k.shape[0]
padding = (0, 0)
dilation = (1, 1)
stride = (1, 1)
# asumiendo que Cin = Cout
Hout = floor((Hin + 2 * padding[0] - dilation[0] * (k1 - 1) - 1) / stride[0]  + 1)
Wout = floor((Win + 2 * padding[1] - dilation[1] * (k2 - 1) - 1) / stride[1]  + 1)

print('Hout: ', Hout)
print('Wout: ', Wout)

# define output shape
out_shape = (N, Cout, Hout, Wout)
print('out_shape: ', out_shape)

Hout:  2
Wout:  2
out_shape:  (1, 3, 2, 2)


strides debe tener una dimension mas que out_shape, ya que cada matrixz $H_{in} \times W_{out}$ se debe dividir en todas las posibles mulitplicaciones del kernel.    

tenemos:

    - [   ,   ,   ,   ,   ] * bytes/itemsize
    
    posicion -1: strides de la convolucional horizontales := stride[1]
    posicion -2: Win
    posicion -3: strides de la convolucional verticales :=  stride[0]
    posicion -4: Hin x Win
    posicion -5: Cin x Hin x Win

y el shape del strided array tiene que ser:
    
    -  [   ,   ,   ,   ,   ]
    
    posicion -1: N
    posicion -2: Wout
    posicion -3: Hout
    posicion -4: k1
    posicion -5: k2

In [113]:
shape_ = (
          N,
          Hout, 
          Wout, 
          Cin,
          k1, 
          k2)
strides_ = np.array([
                    Hin*Win*Cin,
                    Win*stride[0],
                    stride[1], 
                     Hin*Win, 
                     Win*dilation[0], 
                     dilation[1]]) * a.itemsize
print('strides no byte: ', strides_/a.itemsize)
print('strides_: ', strides_)
print('shape_: ', shape_)
print('k shape: ', k.shape)

strides no byte:  [18.  3.  1.  9.  3.  1.]
strides_:  [144  24   8  72  24   8]
shape_:  (1, 2, 2, 2, 2, 2)
k shape:  (3, 1, 2, 2)


In [114]:
strided_array = as_strided(a, shape=shape_, strides=strides_)
strided_array

array([[[[[['a', 'b'],
           ['d', 'e']],

          [['j', 'k'],
           ['m', 'n']]],


         [[['b', 'c'],
           ['e', 'f']],

          [['k', 'l'],
           ['n', 'o']]]],



        [[[['d', 'e'],
           ['g', 'h']],

          [['m', 'n'],
           ['p', 'q']]],


         [[['e', 'f'],
           ['h', 'i']],

          [['n', 'o'],
           ['q', 'r']]]]]], dtype=object)

In [115]:
out = np.tensordot(k, strided_array, axes=([1, 2, 3], [3, 4, 5]))
print(out,'\n')
print('bueno: \n\n', np.swapaxes(out, 0, 1))
print('shape: ', np.swapaxes(out, 0, 1).shape)
print('desired shape: ', out_shape)

ValueError: shape-mismatch for sum